## lab 06-1 : TensorFlow로 Softmax Classification 구현

**Softmax function**  
- n개의 예측할 것이 있을 때 softmax function을 사용하면 좋음  
- Softmax function으로 확률로 변환할 수 있고 이 확률들을 다 더하면 1이 됨  

<br>
<img src = 'https://miro.medium.com/max/1906/1*ReYpdIZ3ZSAPb2W8cJpkBg.jpeg'>  

<br>

**Cost function: cross entropy**  

<img src ='https://t1.daumcdn.net/cfile/tistory/263B2C485797F9C50A'>

In [2]:
import tensorflow as tf

In [8]:
tf.set_random_seed(777) # for repoducibility

x_data = [[1,2,1,1],
         [2,1,3,2],
         [3,1,3,4],
         [4,1,5,5],
         [1,7,5,5],
         [1,2,5,6],
         [1,6,6,6],
         [1,7,7,7]]

y_data = [[0,0,1], # 2번째 자리라서 2 (one-hot encoding)
         [0,0,1], 
         [0,0,1],
         [0,1,0],  # 1번째 자리라서 1
         [0,1,0],
         [0,1,0],
         [1,0,0],
         [1,0,0]]

X = tf.placeholder("float",[None,4])
Y = tf.placeholder("float",[None,3]) # One-hot으로 표시하다 보니 3개 -> label 개수(0,1,2)
nb_classes = 3 # 클래스 개수

W = tf.Variable(tf.random_normal([4,nb_classes],name='weight')) # 4는 x의 값, 나가는 값은 y의 값
b = tf.Variable(tf.random_normal([nb_classes]),name='bias') # 출력 값, y의 값

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits),dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis),axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch grpah
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _, cost_val = sess.run([optimizer,cost],feed_dict={X:x_data, Y:y_data})
        
        if step % 200 == 0:
            print(step, cost_val)
            
    print('------------')
    
    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X:[[1,11,7,9]]})
    print(a,sess.run(tf.argmax(a,1))) # array 중 가장 큰 값

    print('------------')
    b = sess.run(hypothesis, feed_dict={X:[[1,3,4,3]]})
    print(b,sess.run(tf.argmax(b,1)))
    
    print('------------')
    c = sess.run(hypothesis, feed_dict={X:[[1,1,0,1]]})
    print(c,sess.run(tf.argmax(c,1)))
    
    print('------------')
    all = sess.run(hypothesis, feed_dict={X:[[1,11,7,9],[1,3,4,3],[1,1,0,1]]})
    print(all,sess.run(tf.argmax(all,1)))

0 3.4137232
200 0.65390223
400 0.5497438
600 0.45920146
800 0.37088162
1000 0.28212956
1200 0.22390842
1400 0.20385072
1600 0.18697551
1800 0.17258716
2000 0.16018382
------------
[[1.0524965e-02 9.8946381e-01 1.1155750e-05]] [1]
------------
[[0.7977393  0.17022017 0.03204053]] [0]
------------
[[2.0318602e-08 3.6031994e-04 9.9963963e-01]] [2]
------------
[[1.0524965e-02 9.8946381e-01 1.1155750e-05]
 [7.9773957e-01 1.7021997e-01 3.2040507e-02]
 [2.0318602e-08 3.6031994e-04 9.9963963e-01]] [1 0 2]


## lab 06-2 : TensorFlow로 Fancy Softmax Classification 구현

In [15]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7 # 0~6

X = tf.placeholder(tf.float32, [None,16]) # X의 feature가 16개
Y = tf.placeholder(tf.int32, [None,1]) # 0~6 shape=(?,1)

Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape =(?,1,7)
print("one_hot:",Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_classes]) # shape=(?,7)
print("reshape one_hot:", Y_one_hot)

W = tf.Variable(tf.random_normal([16,nb_classes]),name='weight')
b = tf.Variable(tf.random_normal([nb_classes]),name='bias')

# tf.nn.softmax computes softmax activations
# sotfmax = exp(logits) / reduce_sum(exp(logits),dim)
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# Launch grpah
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _,cost_val,acc_val = sess.run([optimizer,cost,accuracy],feed_dict={X:x_data, Y:y_data})
        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))
            
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X:x_data})
    
    # y_data: (N,1) = flatten -> (N,) matches pred.shape
    for p,y in zip(pred, y_data.flatten()): # y가 [[1],[0]] 이런 형태로 묵여 있는 것을 [1,0] 이런 식으로 flatten
                                            # zip은 묶어서 표현
       print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot: Tensor("one_hot_2:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape_2:0", shape=(?, 7), dtype=float32)
Step:     0	Cost: 3.513	Acc: 17.82%
Step:   100	Cost: 0.651	Acc: 83.17%
Step:   200	Cost: 0.426	Acc: 87.13%
Step:   300	Cost: 0.318	Acc: 91.09%
Step:   400	Cost: 0.253	Acc: 93.07%
Step:   500	Cost: 0.211	Acc: 93.07%
Step:   600	Cost: 0.181	Acc: 94.06%
Step:   700	Cost: 0.159	Acc: 96.04%
Step:   800	Cost: 0.141	Acc: 97.03%
Step:   900	Cost: 0.126	Acc: 98.02%
Step:  1000	Cost: 0.114	Acc: 99.01%
Step:  1100	Cost: 0.104	Acc: 99.01%
Step:  1200	Cost: 0.096	Acc: 99.01%
Step:  1300	Cost: 0.088	Acc: 99.01%
Step:  1400	Cost: 0.082	Acc: 99.01%
Step:  1500	Cost: 0.076	Acc: 99.01%
Step:  1600	Cost: 0.071	Acc: 100.00%
Step:  1700	Cost: 0.067	Acc: 100.00%
Step:  1800	Cost: 0.063	Acc: 100.00%
Step:  1900	Cost: 0.059	Acc: 100.00%
Step:  2000	Cost: 0.056	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y

**one-hot**  
- If the input indices is rank N, the output will have rank N+1. The new axis is created at dimension axis